In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

In [3]:
df = pd.read_excel("Motor Trend Car Road Tests.xlsx")
x = np.reshape(df["hp"], [-1,1])
y = np.reshape(df["mpg"], [-1,1])

In [4]:
model = LinearRegression(fit_intercept=True)
model.fit(x, y)

LinearRegression()

In [5]:
def std_errors(X, y, model, degree):

    y_pred = model.predict(X)

    RSS = np.sum((y_pred - y)**2)
    n = len(y)
    p = degree + 1
    mean = X.mean()
    variance = RSS/(n-p)
    RSE = np.sqrt(RSS/(n-p))

    standard_error_b0 = np.sqrt(variance * (1/n + (mean**2 / (np.sum((X - mean)**2)))))
    standard_error_b1 = np.sqrt(variance / (np.sum((X - mean)**2)))
                     
                     
    return RSS, RSE, standard_error_b0, standard_error_b1

In [6]:
RSS, RSE, SEb0, SEb1 = std_errors(x, y, model, 1)

In [7]:
SEb0

1.633920950302906

In [8]:
SEb1

0.010119303810422772

In [9]:
def confidence_intervals(intercept, coef, standard_error_b0, standard_error_b1):

    upper_b0 = intercept + 2*standard_error_b0
    lower_b0 = intercept - 2*standard_error_b0

    upper_b1 = coef + 2*standard_error_b1
    lower_b1 = coef - 2*standard_error_b1

    b0_ci = [lower_b0, upper_b0]
    b1_c1 = [lower_b1, upper_b1]

    return b0_ci, b1_c1

In [10]:
confidence_intervals(model.intercept_, model.coef_, SEb0, SEb1)

([array([26.83101864]), array([33.36670244])],
 [array([[-0.08846689]]), array([[-0.04798967]])])

In [11]:
model.coef_

array([[-0.06822828]])

In [12]:
model.intercept_

array([30.09886054])

In [13]:
b0_estimators = []
b1_estimators = []

for i in range(1000):
    new_df = df.sample(n=len(df), replace=True)
    x = np.reshape(new_df["hp"], [-1,1])
    y = np.reshape(new_df["mpg"], [-1,1])   
    model = LinearRegression(fit_intercept=True)
    model.fit(x, y)
    
    b0_estimator_i = model.intercept_
    b1_estimator_i = model.coef_

    b0_estimators.append(b0_estimator_i)
    b1_estimators.append(b1_estimator_i)
    

In [14]:
std_b0 = np.std(b0_estimators)
std_b1 = np.std(b1_estimators)

In [15]:
std_b0

2.081411256476891

In [16]:
std_b1

0.013725076902409544

In [17]:
mean_b0 = np.mean(b0_estimators)
mean_b1 = np.mean(b1_estimators)

In [18]:
mean_b0

30.316501022117816

In [19]:
mean_b1

-0.07058641693269391

# BOOTSTRAP DE ÁRBOLES

In [51]:
data

,TV,radio,newspaper,sales,TV*RADIO,TV*PERIODICO,RADIO*PERIODICO
0,230.1,37.8,69.2,22.1,8697.78,15922.92,2615.76
1,44.5,39.3,45.1,10.4,1748.85,2006.95,1772.43
2,17.2,45.9,69.3,9.3,789.48,1191.96,3180.87
3,151.5,41.3,58.5,18.5,6256.95,8862.75,2416.05
4,180.8,10.8,58.4,12.9,1952.64,10558.72,630.72
...,...,...,...,...,...,...,...
195,38.2,3.7,13.8,7.6,141.34,527.16,51.06
196,94.2,4.9,8.1,9.7,461.58,763.02,39.69
197,177.0,9.3,6.4,12.8,1646.10,1132.80,59.52
198,283.6,42.0,66.2,25.5,11911.20,18774.32,2780.40


In [36]:
data = pd.read_csv("Advertising.csv")
data = data.drop(columns="Unnamed: 0")
data["TV*RADIO"] = data["TV"] *data["radio"]
data["TV*PERIODICO"] = data["TV"]*data["newspaper"]
data["RADIO*PERIODICO"] = data["radio"]*data["newspaper"]

In [74]:
factors = []
models = []


for i in range(1000):
    new_data =  data.sample(n=len(data), replace=True)
    charac = data.columns
    factors_i = np.random.choice(charac, size=4, replace=False)
    x = new_data[factors_i]
    y = new_data["sales"]

    model_i = LinearRegression(fit_intercept=True)
    model_i.fit(x, y)

    factors.append(factors_i)
    models.append(model_i)




In [99]:
len(models)

1000

In [75]:
predictions = []

for i in range(len(models)):
    x = data[factors[i]]
    y = data["sales"]

    pred_i = models[i].predict(x)

    predictions.append(pred_i)

In [104]:
test = np.array(predictions)

In [112]:
y_pred = test.mean(axis=0)

In [113]:
r2 = r2_score(y, y_pred)
r2

0.9924001952493365

In [114]:
model = LinearRegression(fit_intercept=True)
x = data.drop(columns="sales")
y = data["sales"]

In [115]:
model.fit(x, y)

LinearRegression()

In [117]:
y_pred = model.predict(x)

In [119]:
r2_score(y, y_pred)

0.968631106625557